# Purpose
1. Hold ListWise Ranking Algorthim
2. Train Algorthim
3. Release trained Algorthim as a file

In [5]:
import pprint
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_ranking as tfr
import tensorflow_recommenders as tfrs
import mysql.connector
import pandas as pd
from getpass import getpass
from mysql.connector import connect, Error

# Phase 1: Get (train and test) data

### Retrive User, Book, and Rating data

##### Current Problem
need to get UserAttribute and BookAttribute Data

In [6]:
# Connect to database
try:
    with connect(
        host="localhost",
        user="root",
        password="mysql",
        database="Sprint1BasicEComDb"
    ) as connection:
        print(connection)
except Error as e:
    print(e)

# Get user ratings from database # really should be making this API calls # lets save as CSV files


def get_user_ratings(): #TODO: like get All Books, add attributes to the query
    query = "SELECT userId,bookId,rating FROM UserBookRatings"
    # incase connection is lost, reconnect
    connection.reconnect(attempts=3, delay=5)
    mydb = connection.cursor()
    mydb.execute(query)
    user_ratings = mydb.fetchall()
    return user_ratings


def get_all_books_metadata():
    query =""" 
    """
    # incase connection is lost, reconnect
    connection.reconnect(attempts=3, delay=5)
    mydb = connection.cursor()
    mydb.execute(query)
    all_books = mydb.fetchall()
    return all_books


def get_all_users_metadata():
    query = "SELECT id,birthDate,sex FROM Users"
    # incase connection is lost, reconnect
    connection.reconnect(attempts=3, delay=5)
    mydb = connection.cursor()
    mydb.execute(query)
    all_users = mydb.fetchall()
    return all_users


def save_to_csv(data, filename, header):
    df = pd.DataFrame(data).set_axis(header, axis=1)
    df.to_csv(filename, index=False, )


#save_to_csv(get_user_ratings(), 'user_ratings.csv', ['user_id', 'movie_title', 'rating'])
#save_to_csv(get_all_books(), 'all_books.csv', ['book_id', 'book_title', 'description', 'num_pages', 'rating', 'num_of_voters','genres','formats','authors'])
#save_to_csv(get_all_users(), 'all_users.csv', ['user_id', 'birth_date'])


In [7]:
#Turn User ratings into a dataframe from list of tuples

df = get_user_ratings()
user_ratings_ids = pd.DataFrame(df, columns=["user_id", "movie_title", "user_rating"])

# convert to string
user_ratings_ids['user_id'] = user_ratings_ids['user_id'].astype(str)
user_ratings_ids['movie_title'] = user_ratings_ids['movie_title'].astype(str)

rating_rank = user_ratings_ids[['user_id', 'movie_title', 'user_rating']].copy()
book_rank = user_ratings_ids[['movie_title']].copy()

user_ratings_ids=[]

print(rating_rank)

  

      user_id movie_title  user_rating
0           1           1         0.09
1           2           1         0.17
2           3           1         1.29
3           4           1         0.21
4           5           1         0.19
...       ...         ...          ...
49995      96         500         0.10
49996      97         500         0.02
49997      98         500         0.07
49998      99         500         0.06
49999     100         500         0.03

[50000 rows x 3 columns]


In [8]:
# Create a tf.data.Dataset from the dataframe
# Will cause error if ran again

books = tf.data.Dataset.from_tensor_slices(dict(book_rank)) # book_rank only 
books = books.map(lambda x: x["movie_title"])


rating_rank = tf.data.Dataset.from_tensor_slices(dict(rating_rank))
rating_rank = rating_rank.map(lambda x: {
    'user_id': x['user_id'],
    'movie_title': x['movie_title'],
    'user_rating': x['user_rating'],
    'book_id': x['book_id']
    'book_title': x['book_title'] 
    'num_pages': x['num_pages'],
    'pre_rating': x['pre_rating'],
})

## new 

unique_books = np.unique(np.concatenate(list(books.batch(1000)))) # is it correct IDK # should be strings

unique_user_ids = np.unique(np.concatenate(list(rating_rank.batch(1_000).map(lambda x: x['user_id'])))) ## could be book id

# decode from bytes to string
unique_books = [book.decode('utf-8') for book in unique_books]
unique_user_ids = [user_id.decode('utf-8') for user_id in unique_user_ids]



SyntaxError: invalid syntax. Perhaps you forgot a comma? (3382576429.py, line 13)

In [ ]:
tf.random.set_seed(42)
# do we have 100_000 ratings? # user_ratings needs to be a new type
shuffled = rating_rank.shuffle(
    100_000, seed=42, reshuffle_each_iteration=False)
pprint.pprint(rating_rank)
pprint.pprint(shuffled)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

# We sample 50 lists for each user for the training data. For each list we
# sample 5 movies from the movies the user rated.
train = tfrs.examples.movielens.sample_listwise(
    train,
    num_list_per_user=50,
    num_examples_per_list=5,
    seed=42
)
test = tfrs.examples.movielens.sample_listwise(
    test,
    num_list_per_user=1,
    num_examples_per_list=5,
    seed=42
)


def sample_listwise(
    rating_dataset: tf.data.Dataset,
    num_list_per_user: int = 10,
    num_examples_per_list: int = 10,
    seed: Optional[int] = None,
) -> tf.data.Dataset:
    """Function for converting the MovieLens 100K dataset to a listwise dataset.

    Args:
        rating_dataset:
          The MovieLens ratings dataset loaded from TFDS with features "movie_title", "user_id", and "user_rating".
        num_list_per_user:
          An integer representing the number of lists that should be sampled for each user in the training dataset.
        num_examples_per_list:
          An integer representing the number of movies to be sampled for each list from the list of movies rated by the user.
        seed:
          An integer for creating `np.random.RandomState`.

    Returns:
        A tf.data.Dataset containing list examples.

        Each example contains three keys: "user_id", "movie_title", and "user_rating". "user_id" maps to a string tensor that represents the user 
        id for the example. "movie_title" maps to a tensor of shape [sum(num_example_per_list)] with dtype tf.string. 
        It represents the list of candidate movie ids. "user_rating" maps to a tensor of shape [sum(num_example_per_list)] 
        with dtype tf.float32. It represents the rating of each movie in the candidate list.
    """
    random_state = np.random.RandomState(seed)

    example_lists_by_user = collections.defaultdict(_create_feature_dict)

    movie_title_vocab = set()
    for example in rating_dataset:
        user_id = example["user_id"].numpy()
        example_lists_by_user[user_id]["movie_title"].append(
            example["movie_title"])
        example_lists_by_user[user_id]["user_rating"].append(
            example["user_rating"])
        movie_title_vocab.add(example["movie_title"].numpy())

    tensor_slices = {"user_id": [], "movie_title": [], "user_rating": []}

    for user_id, feature_lists in example_lists_by_user.items():
        for _ in range(num_list_per_user):

            # Drop the user if they don't have enough ratings.
            if len(feature_lists["movie_title"]) < num_examples_per_list:
                continue

            sampled_movie_titles, sampled_ratings = _sample_list(
                feature_lists,
                num_examples_per_list,
                random_state=random_state,
            )
            tensor_slices["user_id"].append(user_id)
            tensor_slices["movie_title"].append(sampled_movie_titles)
            tensor_slices["user_rating"].append(sampled_ratings)

    return tf.data.Dataset.from_tensor_slices(tensor_slices)

# print(f"train: {train}")

<_MapDataset element_spec={'user_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'movie_title': TensorSpec(shape=(), dtype=tf.string, name=None), 'user_rating': TensorSpec(shape=(), dtype=tf.float64, name=None)}>
<_ShuffleDataset element_spec={'user_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'movie_title': TensorSpec(shape=(), dtype=tf.string, name=None), 'user_rating': TensorSpec(shape=(), dtype=tf.float64, name=None)}>


: 

### Pre-Processing

Current Problems:
* int/double features continues or non continous
* NOTHING IS TESTED

In [ ]:
#get all books and users from csv
book_df = pd.read_csv('Book_MadeUp.csv')

# book_title
moive_titles = book_df['movie_titles'].values
movie_titles = list(map(str, moive_titles))
print(movie_titles)
book_description = book_df['book_descriptions'].values
book_num_pages = book_df['num_pages'].values
book_ratings = book_df['ratings'].values
book_genres = book_df['genres'].values# is array/list
book_authors = book_df['authors'].values# is array/list

### unique values # lloks like  no need for unique values###
unique_movie_titles = np.unique(moive_titles)
unique_book_description = np.unique(book_description)
unique_book_num_pages = np.unique(book_num_pages)
unique_book_ratings = np.unique(book_ratings)
unique_book_genres = np.unique(book_genres)


all_genres = ','.join(book_genres).split(',')
book_unique_genres = list(set(all_genres))# should these np.unique  
book_unique_authors = list(set(book_authors))# should these np.unique   

### unique values ###

all_genres = ','.join(book_genres).split(',')





['1', '2', '3', '4', '5', '6', '7']


##### Book Pre-Processing

In [ ]:
# for text

# Book title 
book_title_lookup = tf.keras.layers.StringLookup()
book_title_lookup.adapt(movie_titles)# should be unique book titles, the strings

#book description
book_description_lookup = tf.keras.layers.TextVectorization()
book_description_lookup.adapt(book_description) # should be a list of book descriptions (make sure less than 500mb)

#num pages # not sure honestly
num_pages = tf.keras.layers.Normalization(axis=None)
num_pages.adapt(book_num_pages.astype(np.float32)) # list of all num pages

#rating # think wrong
rating = tf.keras.layers.Normalization(axis=None)
rating.adapt(book_ratings.astype(np.float32)) # list of all ratings # type is double

# authors
book_author_lookup = tf.keras.layers.StringLookup() # WRONG !!!!
book_author_lookup.adapt(book_authors) # list of all authors

# genres
book_genre_lookup = tf.keras.layers.StringLookup()
book_genre_lookup.adapt(book_genres) # list of all genres

In [ ]:
# Movie model # could test ind

class BookModel(tf.keras.Model):

    def __init__(self):
        super().__init__()

        max_tokens = 10_000 #?

        self.book_title_embedding = tf.keras.Sequential([
            book_title_lookup,
            tf.keras.layers.Embedding(book_title_lookup.vocab_size(), 32)
        ])
        
        # I think needs changing # MAYBE LEAVE OUT FOR NOW
        self.book_description_embeddings = tf.keras.Sequential([
            tf.keras.layers.TextVectorization(max_tokens=max_tokens),
            tf.keras.layers.Embedding(max_tokens, 32, mask_zero=True),
            # We average the embedding of individual words to get one embedding vector
            # per title.
            tf.keras.layers.GlobalAveragePooling1D(),
        ])
        
        #num pages
        self.num_pages_embedding = tf.keras.Sequential([
            num_pages
        ])
        
        #rating
        self.rating_embedding = tf.keras.Sequential([
            rating
        ])
        
        # Preferences as categorical features
        self.author_embedding = tf.keras.Sequential([
            book_author_lookup,
            tf.keras.layers.Embedding(book_author_lookup.vocabulary_size(), 32)
        ])

        self.book_genre_embedding = tf.keras.Sequential([
             book_genre_lookup,
            tf.keras.layers.Embedding( book_genre_lookup.vocabulary_size(), 32)
        ])
     

    def call(self, inputs):
        return tf.concat([
            self.book_title_embedding(inputs["movie_titles"]),
            self.book_description_embeddings(inputs["book_descriptions"]),
            self.num_pages_embedding(inputs["num_pages"]),
            self.rating_embedding(inputs["ratings"]),
            self.book_author_embedding(inputs["authors"]),
            self.book_genre_embedding(inputs["genres"])
        ], axis=1)

##### User Pre-Processing

In [ ]:
user_df = pd.read_csv('User_MadeUp.csv')

user_ids = user_df['user_id'].values
user_ids = list(map(str, user_ids))

user_age = user_df['user_age'].values
user_genre_pref = user_df['genres'].values
user_author_pref = user_df['authors'].values
user_sex= user_df['user_sex'].values

all_user_genres = ','.join(user_genre_pref).split(',')
all_author_pref = ','.join(user_author_pref).split(',')
user_age = np.asarray(user_age).astype('float32')

print(user_ids)
print(user_age)
print(all_user_genres)
print(user_sex)
print(all_author_pref)


['1', '2', '3', '4', '5']
[18. 25. 50. 70. 80.]
['classics', 'fiction', 'historical', 'historical fiction', 'italian literature', 'literature', 'mystery', 'childrens', 'classics', 'fantasy', 'fiction', 'juvenile', 'middle grade', 'young adult', 'astronomy', 'history', 'nonfiction', 'philosophy', 'physics', 'science', 'space', 'action', 'adventure', 'espionage', 'fiction', 'mystery', 'suspense', 'thriller', 'childrens', 'classics', 'fantasy', 'fiction', 'juvenile', 'middle grade', 'young adult']
[ True  True  True  True  True]
['robert ludlum', 'lynne reid banks', 'stephen king', 'robert ludlum', 'lynne reid banks', 'stephen king', 'robert ludlum', 'lynne reid banks', 'stephen king', 'robert ludlum', 'lynne reid banks', 'stephen king', 'robert ludlum', 'lynne reid banks', 'stephen king']


In [ ]:

# User ID 
user_id_lookup = tf.keras.layers.StringLookup()
user_id_lookup.adapt(user_ids)

#  User age # TODO: get all user ages as array
user_age_lookup = tf.keras.layers.Normalization(axis=None)
user_age_lookup.adapt(np.array([user_age])) 

# Genre Preference :string[]
genre_lookup = tf.keras.layers.StringLookup()
genre_lookup.adapt(all_user_genres)

# Author Preference :string[]
author_lookup = tf.keras.layers.StringLookup()
author_lookup.adapt(all_author_pref)

# TODO: change data types of 
    # book_length_pref, publication_age_pref

In [ ]:

# NOT TESTED

class UserModel(tf.keras.Model):

    # don't these need to be passed in?
    def __init__(self):
        super().__init__()

        # User embedding based on user_id
        self.user_id_embedding = tf.keras.Sequential([
            user_id_lookup,
            tf.keras.layers.Embedding(user_id_lookup.vocabulary_size(), 32),
        ])

        # Age as a continuous feature, we'll normalize it
        self.age_normalization = tf.keras.layers.Normalization(axis=None)

        # Sex as a boolean feature, could be treated directly as an input or embedded
        self.sex_embedding = tf.keras.layers.Embedding(
            2, 32)  # Assuming sex is represented as 0 or 1

        # Preferences as categorical features
        self.author_embedding = tf.keras.Sequential([
            author_lookup,
            tf.keras.layers.Embedding(author_lookup.vocabulary_size(), 32)
        ])

        self.genre_embedding = tf.keras.Sequential([
            genre_lookup,
            tf.keras.layers.Embedding(genre_lookup.vocabulary_size(), 32)
        ])

    def call(self, inputs):
        # User feature from user ID
        user_id_feature = self.user_id_embedding(inputs["user_id"])

        # Process age
        age_feature = tf.reshape(
            self.age_normalization(inputs["age"]), (-1, 1))

        # Process sex
        sex_feature = self.sex_embedding(tf.cast(inputs["sex"], tf.int32))

        # Process preferences
        author_features = self.author_embedding(inputs["author"])# can't be right input = string[]
        genre_features = self.genre_embedding(inputs["genre"])# can't be right input = string[]

        # Concatenate all features
        return tf.concat([
            user_id_feature,
            age_feature,
            sex_feature,
            author_features,
            genre_features,
            format_features
        ], axis=1)

# Phase 2: Define Model

In [ ]:
class RankingModel(tfrs.Model):

    def __init__(self, loss):
        super().__init__()
        embedding_dimension = 32

        # User embeddings
        self.user_embeddings = UserModel()
        # Restaurand embeddings
        self.book_embeddings = BookModel()
        # Compute predictions
        self.score_model = tf.keras.Sequential([
            # Learn multiple dense layers.
            tf.keras.layers.Dense(128, activation="relu"),
            tf.keras.layers.Dense(64, activation="relu"),
            tf.keras.layers.Dense(1)
        ])

        self.task = tfrs.tasks.Ranking(
            loss=loss,
            metrics=[
                tfr.keras.metrics.NDCGMetric(name="ndcg_metric"),
                tf.keras.metrics.RootMeanSquaredError()
            ]
        )

    def call(self, features):  # features Type: Dict{}, Key: UserID, BookID, Value: Tensor
        user_embeddings = self.user_model({
            'user_id': features['user_id'],
            'age': features['user_age'],  # Adjust field name as necessary
            'sex': features['user_sex'],  # Adjust field name as necessary
            'author': features['author'],  # If applicable
            'genre': features['genre'],  # If applicable
        })

        # Process movie features through the MovieModel
        # Adjust field names and structure as necessary to match your MovieModel's expected input
        book_embeddings = self.movie_model({
            'movie_title': features['movie_title'],
            'book_description': features['book_description'],  # If applicable
            'num_pages': features['num_pages'],  # If applicable
            'rating': features['rating'],  # If applicable
            'author': features['author'],  # If applicable
            'genre': features['genre'],  # If applicable
        })
        
        user_embeddings_expanded = tf.expand_dims(user_embeddings, 1)
    
        combined_embeddings = tf.concat([user_embeddings_expanded, movie_embeddings], axis=-1)

        return self.score_model(combined_embeddings)

    def compute_loss(self, features, training=False):
        labels = features.pop("user_rating")
        scores = self(features)
        return self.task(
            labels=labels,
            predictions=tf.squeeze(scores, axis=-1),
        )

NameError: name 'tfrs' is not defined

# Phase 3
#### Train Model

In [ ]:
print(train)
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()
print (cached_train)

<_TensorSliceDataset element_spec={'user_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'movie_title': TensorSpec(shape=(5,), dtype=tf.string, name=None), 'user_rating': TensorSpec(shape=(5,), dtype=tf.float64, name=None)}>
<CacheDataset element_spec={'user_id': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'movie_title': TensorSpec(shape=(None, 5), dtype=tf.string, name=None), 'user_rating': TensorSpec(shape=(None, 5), dtype=tf.float64, name=None)}>


In [ ]:
listwise_model = RankingModel(tfr.keras.losses.ListMLELoss())
listwise_model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))
listwise_model.fit(cached_train, epochs=100, verbose=True)  # error here

Epoch 1/100


AttributeError: in user code:

    File "c:\Python311\Lib\site-packages\keras\src\engine\training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "c:\Python311\Lib\site-packages\keras\src\engine\training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Python311\Lib\site-packages\keras\src\engine\training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "c:\Python311\Lib\site-packages\tensorflow_recommenders\models\base.py", line 68, in train_step
        loss = self.compute_loss(inputs, training=True)
    File "C:\Users\DZera\AppData\Local\Temp\ipykernel_16956\3841226348.py", line 55, in compute_loss
        scores = self(features)
    File "c:\Python311\Lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\DZera\AppData\Local\Temp\__autogr